<a href="https://colab.research.google.com/github/BioSHED05/BioShed/blob/main/Bio_SHED.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 📦 Install required packages
!pip install scanpy seaborn matplotlib decoupler igraph leidenalg

# 📥 Download and unzip the dataset
!wget ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE72nnn/GSE72857/suppl/GSE72857_umitab.txt.gz
!gunzip GSE72857_umitab.txt.gz

# 🧬 Load the dataset efficiently
import pandas as pd
import scanpy as sc
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Read in chunks using pandas and convert to a sparse matrix directly
# This avoids loading the entire dense dataframe into memory
df_iterator = pd.read_csv("GSE72857_umitab.txt", sep="\t", index_col=0, chunksize=10000)
chunks = []
for chunk in df_iterator:
    chunks.append(chunk)

df = pd.concat(chunks)

# Convert to sparse matrix before creating AnnData object
from scipy.sparse import csr_matrix
adata = sc.AnnData(csr_matrix(df.T.values), obs=pd.DataFrame(index=df.columns), var=pd.DataFrame(index=df.index))

print(adata)

# 🔍 Preprocessing
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

# 📉 Dimensionality reduction
sc.pp.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)
sc.tl.leiden(adata, resolution=0.5)

# 🕒 Pseudotime estimation
# 🧠 Bio-SHED: Symbolic Emergence Index (SEI)
def compute_sei(adata, groupby="leiden"):
    sei_scores = {}
    for group in adata.obs[groupby].unique():
        group_adata = adata[adata.obs[groupby] == group] # Use a new variable for the subset
        expr = group_adata.X.toarray() if hasattr(group_adata.X, "toarray") else group_adata.X
        # Use a small epsilon to avoid log(0) and handle potential zeros after normalization
        entropy = -np.sum((expr / (expr.sum(axis=1, keepdims=True) + 1e-8)) * np.log1p(expr + 1e-8), axis=1)
        sei_scores[group] = np.mean(entropy)
    return pd.Series(sei_scores, name="SEI")

sei = compute_sei(adata)
adata.obs["sei"] = adata.obs["leiden"].map(sei)

# Find the leiden cluster with the minimum average SEI score
min_sei_cluster = sei.idxmin()

# Find the cell with the minimum pseudotime within the minimum SEI cluster
# Since dpt_pseudotime was not computed, we will approximate the root cell
# by finding the cell in the minimum SEI cluster with the smallest index
# as a proxy for early cells. This is a heuristic and might need adjustment
# based on biological knowledge.
root_cell_index = adata[adata.obs["leiden"] == min_sei_cluster].obs_names[0]

# Set the root cell
adata.uns['iroot'] = adata.obs_names.get_loc(root_cell_index)

sc.tl.dpt(adata)

sei.to_csv("sei_scores.csv")
print(sei)

# 🔄 Bio-SHED: Kinetic Transition Detection
def detect_transitions(adata, pseudotime_key="dpt_pseudotime", top_n=100):
    pt = adata.obs[pseudotime_key]
    # Ensure that pt is sorted to calculate gradient correctly
    sorted_indices = np.argsort(pt.values)
    pt_sorted = pt.iloc[sorted_indices]
    expr = adata.X.toarray() if hasattr(adata.X, "toarray") else adata.X
    expr_sorted = expr[sorted_indices, :]

    # Calculate gradient and replace NaNs and inf with 0 or a small number
    d_expr = np.gradient(expr_sorted, pt_sorted.values, axis=0)
    d_expr = np.nan_to_num(d_expr, nan=0.0, posinf=1e-8, neginf=-1e-8)


    kinetic_scores = np.std(d_expr, axis=0)

    # Create a DataFrame and filter out rows with NaN kinetic_scores before sorting
    transitions_df = pd.DataFrame({"gene": adata.var_names, "kinetic_score": kinetic_scores})
    transitions_df = transitions_df.dropna(subset=['kinetic_score'])

    # Sort and select top genes
    top_genes_df = transitions_df.sort_values("kinetic_score", ascending=False).head(top_n)

    return top_genes_df

transitions = detect_transitions(adata)
transitions.to_csv("kinetic_transitions.csv", index=False)
print(transitions.head())

# 🧬 Bio-SHED: Motif Enrichment with DoRothEA
import decoupler as dc

# Load DoRothEA regulons using load_regulon
dorothea = dc.dot.load_regulon('mouse', minsize=5, min_confidence=0.6) # Using mouse as the data is from mouse

emergent_expr = adata[:, transitions["gene"]].to_df().T

# Process emergent_expr in chunks for decoupler to save memory
results, norm_acts = dc.run_mlm(mat=emergent_expr, net=dorothea, source='source', target='target', verbose=True, use_chunked=True, chunk_size=1000)
results.to_csv("dorothea_enrichment.csv")
print(results.head())

# 📊 Visualizations

# 1. SEI on UMAP
sc.pl.umap(adata, color="sei", cmap="plasma", title="Symbolic Emergence Index (SEI) on UMAP")

# 2. Emergent Gene Dynamics Across Pseudotime
adata.obs["pt_bin"] = pd.qcut(adata.obs["dpt_pseudotime"], q=10, labels=False, duplicates='drop') # Add duplicates='drop'
emergent_genes = transitions["gene"]
heatmap_data = adata[:, emergent_genes].to_df().groupby(adata.obs["pt_bin"]).mean()
sns.heatmap(heatmap_data.T, cmap="mako", xticklabels=True)
plt.xlabel("Pseudotime Bin")
plt.ylabel("Emergent Genes")
plt.title("Expression Dynamics of Emergent Genes Across Pseudotime")
plt.show()

# 3. Top Emergent Gene on UMAP
top_gene = transitions["gene"].iloc[0]
sc.pl.umap(adata, color=top_gene, cmap="viridis", title=f"Expression of Emergent Gene: {top_gene}")

# 4. SEI vs Pseudotime
sns.scatterplot(x=adata.obs["dpt_pseudotime"], y=adata.obs["sei"], hue=adata.obs["leiden"], palette="viridis")
plt.xlabel("Pseudotime")
plt.ylabel("SEI")
plt.title("Symbolic Emergence Index Across Differentiation Trajectory")
plt.show()

# 5. Motif Enrichment Bubble Plot
sns.scatterplot(data=results.sort_values("score", ascending=False).head(20),
                x="source", y="score", size=-np.log10(results["p_value"].head(20)),
                hue="score", palette="coolwarm", legend=False)
plt.xticks(rotation=45)
plt.title("Top Enriched TFs in Emergent Genes")
plt.ylabel("Enrichment Score")
plt.show()

# 📁 Download results
# from google.colab import files
# files.download("sei_scores.csv")
# files.download("kinetic_transitions.csv")
# files.download("dorothea_enrichment.csv")

--2025-08-07 21:48:25--  ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE72nnn/GSE72857/suppl/GSE72857_umitab.txt.gz
           => ‘GSE72857_umitab.txt.gz.1’
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 130.14.250.7, 130.14.250.11, 130.14.250.12, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|130.14.250.7|:21... connected.
Logging in as anonymous ... Logged in!
==> SYST ... done.    ==> PWD ... done.
==> TYPE I ... done.  ==> CWD (1) /geo/series/GSE72nnn/GSE72857/suppl ... done.
==> SIZE GSE72857_umitab.txt.gz ... 16413252
==> PASV ... done.    ==> RETR GSE72857_umitab.txt.gz ... done.
Length: 16413252 (16M) (unauthoritative)

GSE72857_umitab.txt 100%[===================>]  15.65M  21.4MB/s    in 0.7s    

2025-08-07 21:48:27 (21.4 MB/s) - ‘GSE72857_umitab.txt.gz.1’ saved [16413252]

gzip: GSE72857_umitab.txt already exists; do you wish to overwrite (y or n)? n
	not overwritten
AnnData object with n_obs × n_vars = 10368 × 27297
7    -1.487575
10   -1.151510
0    -1.

# Task
Explain the error in the selected code. If possible, fix the error and incorporate the changes into the existing code. Otherwise, try to diagnose the error.

## Identify emergent genes

### Subtask:
Extract the list of emergent genes from the previous analysis.


**Reasoning**:
Load the "kinetic_transitions.csv" file into a pandas DataFrame and extract the 'gene' column.



In [ ]:
transitions_df = pd.read_csv("kinetic_transitions.csv")
emergent_genes = transitions_df["gene"].tolist()
print(emergent_genes[:10])

**Reasoning**:
The previous code block failed because the pandas library was not imported. Import pandas and then load the "kinetic_transitions.csv" file into a pandas DataFrame and extract the 'gene' column.



In [ ]:
import pandas as pd
transitions_df = pd.read_csv("kinetic_transitions.csv")
emergent_genes = transitions_df["gene"].tolist()
print(emergent_genes[:10])

**Reasoning**:
The previous code failed because the file "kinetic_transitions.csv" was not found. This is likely because the previous cell that created and saved this file did not execute successfully. I will re-run the code from the previous cell that generates and saves the "kinetic_transitions.csv" file.



In [ ]:
# 🧬 Bio-SHED: Kinetic Transition Detection
def detect_transitions(adata, pseudotime_key="dpt_pseudotime", top_n=100):
    pt = adata.obs[pseudotime_key]
    # Ensure that pt is sorted to calculate gradient correctly
    sorted_indices = np.argsort(pt.values)
    pt_sorted = pt.iloc[sorted_indices]
    expr = adata.X.toarray() if hasattr(adata.X, "toarray") else adata.X
    expr_sorted = expr[sorted_indices, :]
    d_expr = np.gradient(expr_sorted, pt_sorted.values, axis=0)
    kinetic_scores = np.std(d_expr, axis=0)
    top_genes = adata.var_names[np.argsort(kinetic_scores)[-top_n:]]
    return pd.DataFrame({"gene": top_genes, "kinetic_score": kinetic_scores[np.argsort(kinetic_scores)[-top_n:]]})

transitions = detect_transitions(adata)
transitions.to_csv("kinetic_transitions.csv", index=False)
print(transitions.head())

**Reasoning**:
The previous code block failed because the `adata` object was not defined. The `adata` object is created and preprocessed in the original notebook code. I will include the code that creates and preprocesses the `adata` object, computes pseudotime, and then compute and save the kinetic transitions.



In [ ]:
# Load the dataset efficiently using dask
import pandas as pd
import scanpy as sc
import numpy as np

# Read in chunks using pandas and convert to a sparse matrix directly
# This avoids loading the entire dense dataframe into memory
df_iterator = pd.read_csv("GSE72857_umitab.txt", sep="\t", index_col=0, chunksize=10000)
chunks = []
for chunk in df_iterator:
    chunks.append(chunk)

df = pd.concat(chunks)

# Convert to sparse matrix before creating AnnData object
from scipy.sparse import csr_matrix
adata = sc.AnnData(csr_matrix(df.T.values), obs=pd.DataFrame(index=df.columns), var=pd.DataFrame(index=df.index))

# 🔍 Preprocessing
sc.pp.filter_cells(adata, min_genes=200)
sc.pp.filter_genes(adata, min_cells=3)
sc.pp.normalize_total(adata, target_sum=1e4)
sc.pp.log1p(adata)

# 📉 Dimensionality reduction
sc.pp.pca(adata)
sc.pp.neighbors(adata)
sc.tl.umap(adata)
sc.tl.leiden(adata, resolution=0.5)

# 🕒 Pseudotime estimation
# 🧠 Bio-SHED: Symbolic Emergence Index (SEI)
def compute_sei(adata, groupby="leiden"):
    sei_scores = {}
    for group in adata.obs[groupby].unique():
        cells = adata[adata.obs[groupby] == group]
        expr = cells.X.toarray() if hasattr(cells.X, "toarray") else cells.X
        # Use a small epsilon to avoid log(0) and handle potential zeros after normalization
        entropy = -np.sum((expr / (expr.sum(axis=1, keepdims=True) + 1e-8)) * np.log1p(expr + 1e-8), axis=1)
        sei_scores[group] = np.mean(entropy)
    return pd.Series(sei_scores, name="SEI")

sei = compute_sei(adata)
adata.obs["sei"] = adata.obs["leiden"].map(sei)

# Find the leiden cluster with the minimum average SEI score
min_sei_cluster = sei.idxmin()

# Find the cell with the minimum pseudotime within the minimum SEI cluster
# Since dpt_pseudotime was not computed, we will approximate the root cell
# by finding the cell in the minimum SEI cluster with the smallest index
# as a proxy for early cells. This is a heuristic and might need adjustment
# based on biological knowledge.
root_cell_index = adata[adata.obs["leiden"] == min_sei_cluster].obs_names[0]

# Set the root cell
adata.uns['iroot'] = adata.obs_names.get_loc(root_cell_index)

sc.tl.dpt(adata)


# 🔄 Bio-SHED: Kinetic Transition Detection
def detect_transitions(adata, pseudotime_key="dpt_pseudotime", top_n=100):
    pt = adata.obs[pseudotime_key]
    # Ensure that pt is sorted to calculate gradient correctly
    sorted_indices = np.argsort(pt.values)
    pt_sorted = pt.iloc[sorted_indices]
    expr = adata.X.toarray() if hasattr(adata.X, "toarray") else adata.X
    expr_sorted = expr[sorted_indices, :]

    # Calculate gradient and replace NaNs and inf with 0 or a small number
    d_expr = np.gradient(expr_sorted, pt_sorted.values, axis=0)
    d_expr = np.nan_to_num(d_expr, nan=0.0, posinf=1e-8, neginf=-1e-8)

    kinetic_scores = np.std(d_expr, axis=0)

    # Create a DataFrame and filter out rows with NaN kinetic_scores before sorting
    transitions_df = pd.DataFrame({"gene": adata.var_names, "kinetic_score": kinetic_scores})
    transitions_df = transitions_df.dropna(subset=['kinetic_score'])

    # Sort and select top genes
    top_genes_df = transitions_df.sort_values("kinetic_score", ascending=False).head(top_n)

    return top_genes_df

transitions = detect_transitions(adata)
transitions.to_csv("kinetic_transitions.csv", index=False)
print(transitions.head())